In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
from datetime import timedelta  

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis\prolific


In [2]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_trial = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_subject = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_subject.csv') \
    .loc[:, 
            ['run_id', 'prolificID', 
             'birthyear', 'webcam_fps', 'webcam_label', 
             'chosenAmount', 'chosenDelay']] \
   .drop_duplicates() \
   .rename(columns={'chosenAmount': 'bonus_USD',
                    'chosenDelay': 'bonus_delay'})
data_subject['prolificID'] = data_subject['prolificID'].str.strip()

data_prolific = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/prolific/prolific_export.csv') \
    .rename(columns={'participant_id': 'prolificID'}) \
    .merge(data_subject,
           on='prolificID', 
           how='left')


overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_trial)],
        [len(data_subject)],
        [len(data_prolific)]
    ], 
    columns=['length'],
    index=['data_et', 
           'data_trial',
           'data_subject', 
           'data_prolific'])
print(overview)

                length
data_et        1000191
data_trial       46932
data_subject       115
data_prolific       51


# Approve subjects

In [3]:
if 'trial_length' in data_prolific.columns: 
    data_prolific = data_prolific.drop(columns='trial_length')

output = []
for subject in data_trial['run_id'].unique():
    prolificID = data_trial.loc[data_trial['run_id']==subject, 'prolificID'].unique()[0]
    trial_length = len(data_trial.loc[data_trial['run_id']==subject, 'trial_index'].unique())
    output.append([subject, prolificID, trial_length])
output = pd.DataFrame(output,
             columns=['run_id', 'prolificID', 'trial_length']
            )

data_prolific = data_prolific.merge(
    output, 
    on=['run_id', 'prolificID'], 
    how='left')

In [4]:
data_et = data_et.merge(
    data_trial.loc[:, ['run_id', 'chinFirst', 'trial_index', 'trial_type', 'task_nr']], 
    on=['run_id', 'trial_index'], 
    how='left'
)

In [5]:
output = []
for subject in data_et['run_id'].unique():
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].count()

    m_count_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].count()
    
    n_choseTop = sum(data_trial.loc[
        (data_trial['trial_type']=='eyetracking-choice') &
        (data_trial['run_id']==subject),
        'key_press']==38)

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   n_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'n_choseTop'])
output

data_prolific = data_prolific.merge(output, on='run_id', how='left')

# These subjects await review

In [6]:
data_prolific['status'].unique()

array(['APPROVED', 'TIMED-OUT', 'RETURNED'], dtype=object)

In [7]:
data_prolific.loc[
    data_prolific['prolificID']=='5c95970cd676900016e1a940', 
    ['run_id', 'prolificID', 'trial_length',
     'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
     'n_choseTop']
].sort_values(by='prolificID')

,run_id,prolificID,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,n_choseTop
19,183.0,5c95970cd676900016e1a940,NaN,0.486199,0.542439,0.469404,0.571782,108.0,231.0,23.0


In [8]:
data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'trial_length',
     'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
     'n_choseTop']
].sort_values(by='prolificID')

,run_id,prolificID,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,n_choseTop


In [9]:
data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    'prolificID'].to_csv(header=None, index=False, line_terminator=', ')

''

In [10]:
data_et.loc[
    (data_et['run_id']==162) & 
    (data_et['trial_type']=='eyetracking-choice'), :]

,x,y,t,t_task,run_id,trial_index,chinFirst,trial_type,task_nr
212889,0.550795,0.441494,540293.0,0.0,162.0,154.0,1.0,eyetracking-choice,2.0
212890,0.553333,0.479946,540352.0,59.0,162.0,154.0,1.0,eyetracking-choice,2.0
212891,0.547378,0.568551,540465.0,172.0,162.0,154.0,1.0,eyetracking-choice,2.0
212892,0.547899,0.658305,540502.0,209.0,162.0,154.0,1.0,eyetracking-choice,2.0
212893,0.549370,0.646920,540599.0,306.0,162.0,154.0,1.0,eyetracking-choice,2.0
...,...,...,...,...,...,...,...,...,...
216943,0.355960,0.562455,946838.0,1348.0,162.0,392.0,1.0,eyetracking-choice,2.0
216944,0.447455,0.498945,946950.0,1460.0,162.0,392.0,1.0,eyetracking-choice,2.0
216945,0.540851,0.464219,946983.0,1493.0,162.0,392.0,1.0,eyetracking-choice,2.0
216946,0.606543,0.456996,947068.0,1578.0,162.0,392.0,1.0,eyetracking-choice,2.0


In [11]:
data_prolific.loc[
    data_prolific['prolificID'].isin(
    ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
         '5edc20443467e28ec4e30f93',
         '5fa1192cf99e161a5cfad1cd', 
         '5d430fdf871f1700017b2a81'
        ]
    ), 
    ['run_id', 'prolificID', 'trial_length',
     'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
     'n_choseTop']
].sort_values(by='prolificID')

,run_id,prolificID,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,n_choseTop


In [12]:
if 'choice_rt' in data_prolific.columns:
    data_prolific = data_prolific.drop(columns='choice_rt')
grouped = data_trial \
    .loc[
        data_trial['trial_type']=='eyetracking-choice', 
        :] \
    .groupby(['prolificID'])['rt'].mean() \
    .reset_index() \
    .rename(columns={'rt': 'choice_rt'})

data_prolific = data_prolific.merge(
    grouped, 
    on='prolificID', 
    how='left')

data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'webcam_fps', 
     'choice_rt', 'bonus_USD', 'bonus_delay']
] \
    .sort_values(by='run_id')

,run_id,prolificID,webcam_fps,choice_rt,bonus_USD,bonus_delay


# data_pay

In [13]:
data_pay = data_prolific.loc[
    data_prolific['status']=='APPROVED', 
    [
        'run_id', 'prolificID', 
        'age',  'Country of Birth', 'Current Country of Residence', 'First Language',
        'Nationality', 'Sex',
        'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
        'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
        'bonus_USD', 'bonus_delay'
    ]
]
print(len(data_pay))

40


In [14]:
data_pay.loc[pd.isna(data_pay['bonus_delay']), ['run_id', 'prolificID', 'bonus_delay', 'status']]

,run_id,prolificID,bonus_delay,status
19,183.0,5c95970cd676900016e1a940,NaN,APPROVED


# Bonus payment

Reformat payments

In [15]:
data_pay['bonus_delay'] = data_pay['bonus_delay'].astype(str)
data_pay['bonus_delay'] = data_pay['bonus_delay'] \
    .replace(['Today', 'Tomorrow', '7 days', 
          '15 days', '30 days', '90 days', 
          '180 days'], 
         [0, 1, 7, 15, 30, 90, 180]) \
    .astype(float)

data_pay['bonus_USD'] = data_pay['bonus_USD'].astype(str)
data_pay['bonus_USD'] = data_pay['bonus_USD'] \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) \
    .astype(float)

data_pay.loc[:, ['bonus_USD', 'bonus_delay']].head(5)

,bonus_USD,bonus_delay
0,4.0,7.0
2,5.0,180.0
4,5.0,7.0
5,4.0,0.0
6,5.0,180.0


## Replace empty values with the best bonus reward

In [16]:
# For two subjects, the bonus payment was not recorded
data_pay.loc[
    data_pay['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
         '5edc20443467e28ec4e30f93',
         '5fa1192cf99e161a5cfad1cd', 
         '5d430fdf871f1700017b2a81',
         '5c95970cd676900016e1a940',
        ]), ['bonus_USD', 'bonus_delay']] = [5, 1]

data_pay.loc[data_pay['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
         '5edc20443467e28ec4e30f93', 
         '5d430fdf871f1700017b2a81',
         '5c95970cd676900016e1a940'
        ]), ['run_id', 'prolificID', 'bonus_USD', 'bonus_delay']]

,run_id,prolificID,bonus_USD,bonus_delay
19,183.0,5c95970cd676900016e1a940,5.0,1.0


In [17]:
data_pay.loc[data_pay[['bonus_USD', 'bonus_delay']].isnull().any(axis=1), :]

,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,Country of Birth,entered_code,session_id,started_datetime,completed_date_time,time_taken,bonus_USD,bonus_delay


## Bonus in other currencies

In [18]:
data_pay['bonus_GBP'] = data_pay['bonus_USD'] * 0.75
data_pay['bonus_EUR'] = data_pay['bonus_GBP'] * 1.13

## When is the bonus due?

In [19]:
def add_completed_date(data, data_trial):
    output = []

    for subject in data_trial['run_id'].unique():
        thisSubject = data_trial.loc[data_trial['run_id']==subject] \
            .reset_index(drop=True)
        date_time_obj = datetime.datetime.strptime(
            thisSubject.loc[0, 'recorded_at'], '%Y-%m-%d %H:%M:%S')

        output.append([thisSubject.loc[0, 'run_id'], date_time_obj.date()])
        
    output = pd.DataFrame(output, columns=['run_id', 'date']) 
    
    if 'date' in data.columns: data = data.drop(columns=['date'])
    data = data.merge(output, on='run_id', how='left')
    return data

data_pay = add_completed_date(data_pay, data_trial)
    
data_pay['completed_date']=data_pay.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)

data_pay['due_on'] = data_pay['completed_date'] + \
    data_pay['bonus_delay'].map(datetime.timedelta) 

print(len(data_pay))
data_pay.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Sex',  
                   'bonus_USD', 'bonus_GBP', 'bonus_EUR', 
                   'completed_date', 'bonus_delay', 'due_on'
               ]
              ]. sort_values(by='due_on')


40


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1342: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  warnings.warn(


,prolificID,run_id,Nationality,Sex,bonus_USD,bonus_GBP,bonus_EUR,completed_date,bonus_delay,due_on
39,5e992379f1bb6715d48226e0,212.0,United States,Male,4.0,3.000,3.39000,2021-02-11,0.0,2021-02-11
34,5ef659991d975d12c6e7c555,208.0,United States,Male,3.5,2.625,2.96625,2021-02-11,0.0,2021-02-11
3,5ead6b0e96a62440399d8caf,151.0,Nepal,Female,4.0,3.000,3.39000,2021-02-11,0.0,2021-02-11
5,5f6560fd86ca9f0ac37ca381,153.0,Bangladesh,Male,3.5,2.625,2.96625,2021-02-11,0.0,2021-02-11
29,58ae59babb9e0f0001acb414,206.0,United States,Male,3.5,2.625,2.96625,2021-02-11,0.0,2021-02-11
26,5f273f8ba9a38c44acd49ffe,197.0,United States,Female,4.0,3.000,3.39000,2021-02-11,0.0,2021-02-11
38,5f48002958dd331635f1d767,211.0,United States,Female,3.5,2.625,2.96625,2021-02-11,0.0,2021-02-11
10,5e6002b9e9e2ca34e24deab8,163.0,United States,Male,1.0,0.750,0.84750,2021-02-11,0.0,2021-02-11
11,5d7fd2a5987f70001602d95b,164.0,Cambodia,Female,2.5,1.875,2.11875,2021-02-11,0.0,2021-02-11
18,5fb5e070ad83631f08d83322,177.0,United States,Female,3.0,2.250,2.54250,2021-02-11,0.0,2021-02-11


## What bonus is due today? 

In [34]:
bonus_due_today = data_pay.loc[data_pay['due_on']==datetime.datetime.now().date(), :]
bonus_due_today['bonus_GBP'] = bonus_due_today['bonus_GBP'].round(2)
bonus_due_today

<ipython-input-34-d6dc15fe434a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonus_due_today['bonus_GBP'] = bonus_due_today['bonus_GBP'].round(2)


,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,...,completed_date_time,time_taken,bonus_USD,bonus_delay,bonus_GBP,bonus_EUR,date,completed_date,due_on,basic_GBP
15,183.0,5c95970cd676900016e1a940,42,Ethiopia,United States,English,Germany,Female,APPROVED,2021-02-11 19:52:07.431000,...,2021-02-11 19:52:06.529000,9945.704,5.0,1.0,3.75,4.2375,2021-02-11,2021-02-11,2021-02-12,2.25
37,213.0,5dc0ac2c441a2105cbb3bb0f,33,United States,United States,English,United States,Male,APPROVED,2021-02-12 08:08:30.701000,...,2021-02-11 22:39:26.156000,1581.690,5.0,1.0,3.75,4.2375,2021-02-11,2021-02-11,2021-02-12,2.25


In [35]:
bonus_due_today.loc[: , ['prolificID', 'bonus_GBP']] \
    .to_csv('C:/Users/User/GitHub/WebET_Analysis/prolific/bonusToday.csv', 
            index=False, 
            header=False
           )
print(bonus_due_today['due_on'].unique())

[datetime.date(2021, 2, 12)]


## Bonus left to pay

In [22]:
data_pay.columns

Index(['run_id', 'prolificID', 'age', 'Country of Birth',
       'Current Country of Residence', 'First Language', 'Nationality', 'Sex',
       'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
       'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
       'bonus_USD', 'bonus_delay', 'bonus_GBP', 'bonus_EUR', 'date',
       'completed_date', 'due_on'],
      dtype='object')

In [23]:
data_pay.loc[
    data_pay['due_on']>datetime.datetime.now().date(), 
    ['prolificID', 'completed_date', 'bonus_GBP', 'due_on']]

,prolificID,completed_date,bonus_GBP,due_on
0,5e9c8aace6eae81abf309d53,2021-02-11,3.000,2021-02-18
1,60220f3c276dd72c78daa7bb,2021-02-11,3.750,2021-08-10
2,5de80e0b521f95791f912e49,2021-02-11,3.750,2021-02-18
4,5ff6841f306c942319b20520,2021-02-11,3.750,2021-08-10
6,5a9c91196219a30001f534a5,2021-02-11,3.750,2021-03-13
7,5ff7baf4cbc6b34ca5779e90,2021-02-11,3.000,2021-02-26
8,5e331de721bbd22ed15f2405,2021-02-11,3.375,2021-02-18
9,5f493291e3e64832df458323,2021-02-11,3.750,2021-02-26
12,5df967d2f6191f6e87b236e6,2021-02-11,3.750,2021-08-10
14,5e8bfc63e53da90ec5673f11,2021-02-11,3.750,2021-02-18


In [24]:
left_bonus = data_pay.loc[
    data_pay['due_on']>datetime.datetime.now().date(), 
    'bonus_GBP'].sum()

left_bonus * 1.2 + (left_bonus*0.2*0.2)

115.32

# Total costs

## Basic payment

In [25]:
data_pay['basic_GBP'] = 2.25
data_pay.loc[:, ['basic_GBP', 'bonus_GBP']].head(5)

,basic_GBP,bonus_GBP
0,2.25,3.00
1,2.25,3.75
2,2.25,3.75
3,2.25,3.00
4,2.25,3.75


## Total

In [26]:
print('N =' + str(len(data_pay)) + '\n')

reward_basic = data_pay['basic_GBP'].sum()
service_fee = reward_basic * 0.2
vat = reward_basic * 0.2*0.2
total_basic = data_pay['basic_GBP'].sum() * 1.4
total_bonus = data_pay['bonus_GBP'].sum() * 1.4
total = total_basic + total_bonus

overview = pd.DataFrame(
    [
        [reward_basic], 
        [service_fee], [vat], 
        [total_basic], [total_bonus], [total]
    ], 
    columns=['GBP'],
    index= ['reward_basic', 
        'service_fee', 'vat', 
        'total_basic', 'total_bonus', 'total'])
print(str(overview) + '\n')

print('Total in €, incl. transaction fee: ' + str(total * 1.14))

N =40

                GBP
reward_basic   90.0
service_fee    18.0
vat             3.6
total_basic   126.0
total_bonus   176.4
total         302.4

Total in €, incl. transaction fee: 344.73599999999993


## Total already paid 
Basic + bonus until today. Use that to compare with prolific bills

In [27]:
bonus_GBP_already_paid = data_pay.loc[
    data_pay['due_on']<=datetime.datetime.now().date(), 
    'bonus_GBP'].sum()

reward_basic = data_pay['basic_GBP'].sum()
service_fee = (reward_basic + bonus_GBP_already_paid) * 0.2
vat = (reward_basic + bonus_GBP_already_paid) * 0.2*0.2
total_basic = data_pay['basic_GBP'].sum() * 1.4
total_bonus = bonus_GBP_already_paid * 1.4
total = total_basic + total_bonus

overview = pd.DataFrame(
    [
        [reward_basic], 
        [service_fee], [vat], 
        [total_basic], [total_bonus], [total]
    ], 
    columns=['GBP'],
    index= ['reward_basic', 
        'service_fee', 'vat', 
        'total_basic', 'total_bonus', 'total'])
print(str(overview) + '\n')

print('Total in €, incl. transaction fee: ' + str(total * 1.14))

                 GBP
reward_basic   90.00
service_fee    24.60
vat             4.92
total_basic   126.00
total_bonus    46.20
total         172.20

Total in €, incl. transaction fee: 196.30799999999996


# Prognosis for the larger sample

In [28]:
def cost_prognosis(n):
    reward_basic = n * data_pay['basic_GBP'].mean()
    reward_bonus = n * data_pay['bonus_GBP'].mean()
        
    total_reward = reward_basic + reward_bonus
    service_fee  = total_reward * 0.2
    vat          = total_reward * 0.2*0.2
    total_basic  = reward_basic + reward_basic*0.2 + reward_basic*0.2*0.2
    total_bonus  = reward_bonus + reward_bonus*0.2 + reward_bonus*0.2*0.2
    total        = total_reward + service_fee + vat

    overview = pd.DataFrame(
        [
            [reward_basic], [reward_bonus], [total_reward],
            [service_fee], [vat], 
            [total_basic], [total_bonus], [total], [total * 1.14]
        ], 
        columns=[n],
        index= ['reward_basic', 'reward_bonus', 'total_reward',
            'service_fee', 'vat', 
            'total_basic', 'total_bonus', 'total GBP', 'total EUR'])
    overview = round(overview, 2)
    return(overview)

print('Budget: ' + str(round(2101*0.88, 2)) + ' GBP / ' + 
      '2101.00 EUR / ' +
      'n=' + str(int(np.floor((2101*0.88)/6.6)))
     )

print('Paid:   1700.50 GBP / ' + 
      str(round(1700.5*1.15, 2)) + ' EUR / ' + 
      'n=' + str(int(np.floor(1700.5/6.6))))
buffer_EUR = 2101-1955.57
buffer_GBP = buffer_EUR * 0.88
print('Buffer: ' + str(round(buffer_GBP, 2)) + ' /  ' + 
      str(round(buffer_EUR, 2)) + ' EUR / n=' + 
      str(int(np.floor(buffer_GBP/6.6))) + '\n')
print('Collect 207 more participants.')

pd.concat([
    cost_prognosis(1),
    cost_prognosis(50),
    cost_prognosis(257), 
    cost_prognosis(207)
], axis=1)

Budget: 1848.88 GBP / 2101.00 EUR / n=280
Paid:   1700.50 GBP / 1955.57 EUR / n=257
Buffer: 127.98 /  145.43 EUR / n=19

Collect 207 more participants.


,1,50,257,207
reward_basic,2.25,112.50,578.25,465.75
reward_bonus,3.15,157.50,809.55,652.05
total_reward,5.40,270.00,1387.80,1117.80
service_fee,1.08,54.00,277.56,223.56
vat,0.22,10.80,55.51,44.71
total_basic,2.79,139.50,717.03,577.53
total_bonus,3.91,195.30,1003.84,808.54
total GBP,6.70,334.80,1720.87,1386.07
total EUR,7.63,381.67,1961.79,1580.12


In [29]:
1409/6.6

213.4848484848485

# Export modified prolific data

In [30]:
data_pay.to_csv('C:/Users/User/GitHub/WebET_Analysis/prolific/data_pay.csv', index=False, header=True)

# Feedback

In [31]:
print('Success! Script ran through.')

Success! Script ran through.
